### SOURCE TABLE 

In [0]:
CREATE TABLE workspace.default.scdtype1_source
(
prod_id INT,
product_name STRING,
product_cat STRING,
process_date DATE



)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7045690484515143>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'CREATE TABLE workspace.default.scdtype1_source\n(\nprod_id INT,\nproduct_name STRING,\nproduct_cat STRING,\nprocess_date DATE\n\n\n\n)\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:192, in SqlMagic.

In [0]:
SELECT * FROM workspace.default.scdtype1_source

prod_id,product_name,product_cat,process_date


In [0]:
INSERT INTO workspace.default.scdtype1_source
VALUES 
  (1, 'prod1', 'cat1', CURRENT_DATE()),
  (2, 'prod2', 'cat2', CURRENT_DATE()),
  (3, 'prod3', 'cat3', CURRENT_DATE())






num_affected_rows,num_inserted_rows
3,3


### TARGET TABLE 

In [0]:
CREATE TABLE workspace.dwh_gold.scdtype1_target
(
prod_id INT,
product_name STRING,
product_cat STRING,
process_date DATE



)


### SOURCE TABLE 

In [0]:
%python
spark.sql("select * from workspace.default.scdtype1_source").createOrReplaceTempView("src")

### MERGING TARGET AND SOURCE TABLE 

In [0]:
MERGE INTO workspace.dwh_gold.scdtype1_target t
USING workspace.default.scdtype1_source s
ON t.prod_id = s.prod_id
WHEN MATCHED AND s.process_date >= t.process_date THEN  -- update only if the new record is newer than the existing one 
UPDATE SET *
WHEN NOT MATCHED THEN INSERT *


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,3,0,0


In [0]:
SELECT * FROM workspace.dwh_gold.scdtype1_target

prod_id,product_name,product_cat,process_date
1,prod1,cat1,2026-02-16
2,prod2,cat2,2026-02-16
3,prod3,new_cat,2026-02-16


In [0]:
UPDATE workspace.default.scdtype1_source SET product_cat= 'new_cat'
WHERE prod_id= 3 

num_affected_rows
1


### SCD TYPE 02

### SOURCE TABLE 

In [0]:
CREATE OR REPLACE TABLE workspace.default.scdtype2_source
(
prod_id INT,
product_name STRING,
product_cat STRING,
process_date DATE



)

In [0]:
INSERT INTO workspace.default.scdtype2_source
VALUES 
  (1, 'prod1', 'cat1', CURRENT_DATE()),
  (2, 'prod2', 'cat2', CURRENT_DATE()),
  (3, 'prod3', 'cat3', CURRENT_DATE())



num_affected_rows,num_inserted_rows
3,3


### TARGET TABLE 

In [0]:
CREATE TABLE workspace.dwh_gold.scdtype2_target
(
prod_id INT,
product_name STRING,
product_cat STRING,
process_date DATE,
end_date DATE,
is_current BOOLEAN
)


In [0]:
SELECT *,
         date(current_timestamp) as start_date,
         CAST('3000-01-01' AS TIMESTAMP) as end_date,
         true as is_currentdate

 FROM workspace.default.scdtype2_source

prod_id,product_name,product_cat,process_date,start_date,end_date,is_currentdate
1,prod1,cat1,2026-02-16,2026-02-16,3000-01-01T00:00:00.000Z,true
2,prod2,cat2,2026-02-16,2026-02-16,3000-01-01T00:00:00.000Z,true
3,prod3,cat3,2026-02-16,2026-02-16,3000-01-01T00:00:00.000Z,true


In [0]:
%python
spark.sql("""SELECT *,
         date(current_timestamp) as start_date,
         CAST('3000-01-01' AS TIMESTAMP) as end_date,
         true as is_currentdate

 FROM workspace.default.scdtype2_source""").createOrReplaceTempView("src")

In [0]:
SELECT * FROM src 

prod_id,product_name,product_cat,process_date,start_date,end_date,is_currentdate
1,prod1,cat1,2026-02-16,2026-02-16,3000-01-01T00:00:00.000Z,true
2,prod2,cat2,2026-02-16,2026-02-16,3000-01-01T00:00:00.000Z,true
3,prod3,cat3,2026-02-16,2026-02-16,3000-01-01T00:00:00.000Z,true


In [0]:
MERGE INTO workspace.dwh_gold.scdtype2_target t
USING src
ON src.prod_id = t.prod_id AND 
t.is_current = true

-- WHEN WE HAVE NEW DATA WITH UPDATES 
WHEN MATCHED AND (
  src.product_name <> t.product_name OR 
  src.product_cat <> t.product_cat OR 
  src.process_date <> t.process_date
)
THEN UPDATE SET
  t.end_date = current_timestamp(),
  t.is_current = false

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,1,0,0


In [0]:
MERGE INTO workspace.dwh_gold.scdtype2_target t
USING src
ON src.prod_id = t.prod_id AND 
t.is_current = true

-- WHEN WE HAVE NEW DATA WITH UPDATES
WHEN NOT MATCHED THEN INSERT 
(
prod_id,
product_name,
product_cat,
process_date,
end_date,
is_current
)
VALUES
(
src.prod_id,
src.product_name,
src.product_cat,
src.process_date,
src.end_date,
src.is_currentdate
)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,0,0,1


In [0]:
DROP TABLE workspace.dwh_gold.scdtype2_target

In [0]:
SELECT * FROM workspace.dwh_gold.scdtype2_target

prod_id,product_name,product_cat,process_date,end_date,is_current
1,prod1,cat1,2026-02-16,3000-01-01,true
3,prod3,cat3,2026-02-16,3000-01-01,true
2,prod2,cat2,2026-02-16,2026-02-16,false
2,prod2,new_cat,2026-02-16,3000-01-01,true


### UPDATING SOURCE TABLE WITH NEW DATA

In [0]:
UPDATE workspace.default.scdtype2_source 
SET product_cat = 'new_cat'
WHERE prod_id = 2

num_affected_rows
1
